The colab notebook contains implemented code based on the following article for music generation
https://www.analyticsvidhya.com/blog/2020/01/how-to-perform-automatic-music-generation/

In [ ]:
from music21 import *
import numpy as np
import os
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import *
from keras.callbacks import *
import keras.backend as K

K.clear_session()

def read_midi(file):
    
    print("Loading Music File:",file)
    
    notes=[]
    notes_to_parse = None
    
    #parsing a midi file
    midi = converter.parse(file)
  
    #grouping based on different instruments
    s2 = instrument.partitionByInstrument(midi)

    #Looping over all the instruments
    for part in s2.parts:
    
        #select elements of only piano
        if 'Piano' in str(part): 
        
            notes_to_parse = part.recurse() 
      
            #finding whether a particular element is note or a chord
            for element in notes_to_parse:
                
                #note
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                
                #chord
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(notes)

path = "/content/drive/MyDrive/ma2/"
files = [i for i in os.listdir(path) if i.endswith(".midi")]
notes_array = np.array([read_midi(path+i) for i in files], dtype = object)
notes = [element for note in notes_array for element in note]

unique_notes = list(set(notes))
freq = dict(Counter(notes))
no = [count for _,count in freq.items()]

frequent_notes = [note for note, count in freq.items() if count>=50]

new_music = []

for n in notes_array:
    temp=[]
    for no in n:
        if no in frequent_notes:
            temp.append(no)            
    new_music.append(temp)
    
new_music = np.array(new_music, dtype = object)

no_of_timesteps = 32
x = []
y = []

for n in new_music:
    for i in range(0, len(n) - no_of_timesteps, 1):
        
        #preparing input and output sequences
        inp = n[i:i + no_of_timesteps]
        out = n[i + no_of_timesteps]
        
        x.append(inp)
        y.append(out)
        
x = np.array(x)
y = np.array(y)

unique_x = list(set(x.ravel()))
x_note_to_int = dict((note, num) for num, note in enumerate(unique_x))

#preparing input sequences
x_seq=[]
for i in x:
    temp=[]
    for j in i:
        #assigning unique integer to every note
        temp.append(x_note_to_int[j])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)

unique_y = list(set(y))
y_note_to_int = dict((note, num) for num, note in enumerate(unique_y)) 
y_seq = np.array([y_note_to_int[i] for i in y])
x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

# model = Sequential()
    
# #embedding layer
# model.add(Embedding(len(unique_x), 100, input_length = 32,trainable = True)) 

# model.add(Conv1D(64,3, padding = 'causal', activation = 'relu'))
# model.add(Dropout(0.2))
# model.add(MaxPool1D(2))
    
# model.add(Conv1D(128,3, activation = 'relu', dilation_rate = 2, padding = 'causal'))
# model.add(Dropout(0.3))
# model.add(MaxPool1D(2))

# model.add(Conv1D(256,3, activation = 'relu', dilation_rate = 4, padding = 'causal'))
# model.add(Dropout(0.3))
# model.add(MaxPool1D(2))
          
# #model.add(Conv1D(256,5,activation='relu'))    
# model.add(GlobalMaxPool1D())
    
# model.add(Dense(256, activation = 'relu'))
# model.add(Dense(len(unique_y), activation = 'softmax'))
    
# model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model = Sequential()
model.add(Embedding(len(unique_x), 100, input_length = 32,trainable = True))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only = True, verbose = 1)
history = model.fit(np.array(x_tr), np.array(y_tr), batch_size = 128, epochs = 150, validation_data = (np.array(x_val), np.array(y_val)), verbose = 1, callbacks = [mc], )

#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

In [ ]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
import keras.backend as K

model = Sequential()
    
#embedding layer
model.add(Embedding(len(unique_x), 100, input_length=32,trainable=True)) 

model.add(Conv1D(64,3, padding='causal',activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
    
model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
          
#model.add(Conv1D(256,5,activation='relu'))    
model.add(GlobalMaxPool1D())
    
model.add(Dense(256, activation='relu'))
model.add(Dense(len(unique_y), activation='softmax'))
    
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only = True, verbose=1)
history = model.fit(np.array(x_tr),np.array(y_tr),batch_size=128,epochs=50, validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

#loading best model
from keras.models import load_model
model = load_model('best_model.h5')